In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import iplot, init_notebook_mode

# Import data
dt={'Record ID': int, 'Agency Code':object, 'Agency Name':object, 'Agency Type':object, 'City':object,
       'State':object, 'Year':object, 'Month':object, 'Incident':object, 'Crime Type':object, 'Crime Solved':object,
       'Victim Sex':object, 'Victim Age':int, 'Victim Race':object, 'Victim Ethnicity':object,
       'Perpetrator Sex':object, 'Perpetrator Age':object, 'Perpetrator Race':object,
       'Perpetrator Ethnicity':object, 'Relationship':object, 'Weapon':object, 'Victim Count':int,
       'Perpetrator Count':int, 'Record Source':object}
df=pd.read_csv("../input/database.csv",dtype=dt)

Correlations
============

Let's have a look if there are any major correlations in the data.
The first striking observation is that the solving of a crime is correlated to perpetrator age, race and ethnicity. In a lesser way, the correlation stays true with the victim attributes.

In [2]:
#Prepare data for correlation analysis.
drop_elements = ['Record ID', 'Agency Code', 'Agency Name', 'Agency Type', 'City', 'State', 'Record Source']
d = df.drop(drop_elements,axis=1)

d['Crime Solved']=d['Crime Solved'].map({"Yes":1,"No":0})
d['Weapon']=d['Weapon'].map({'Blunt Object':0, 'Strangulation':1, 'Unknown':-1, 'Rifle':3, 'Knife':4, 'Firearm':5, 'Shotgun':6, 'Fall':7, 'Handgun':8, 'Drowning':9, 'Suffocation':10, 'Explosives':11, 'Fire':12,'Drugs':13, 'Gun':14, 'Poison':15})
d['Perpetrator Age']=pd.to_numeric(d['Perpetrator Age'], errors='coerce')
d['Perpetrator Sex'] = d['Perpetrator Sex'].map( {'Female': 0, 'Unknown': -1,'Male': 1} ).astype(int)
d['Victim Sex'] = d['Victim Sex'].map( {'Female': 0, 'Unknown': -1,'Male': 1} ).astype(int)
d['Month'] = d['Month'].map( {'January': 1, 'February': 2,'March': 3, 'April':4,'May':5,'June':6,'July':7,"August":8,"September":9,"October":10,"November":11,"December":12} ).astype(int)
d['Victim Ethnicity'] = d['Victim Ethnicity'].map({'Unknown':-1,'Not Hispanic':0,'Hispanic':1})
d['Perpetrator Ethnicity'] = d['Perpetrator Ethnicity'].map({'Unknown':-1,'Not Hispanic':0,'Hispanic':1})
d['Perpetrator Race'] = d['Perpetrator Race'].map({'Unknown':-1,'Native American/Alaska Native':1,'White':2, 'Black':3, 'Asian/Pacific Islander':4})
d['Victim Race'] = d['Victim Race'].map({'Unknown':-1,'Native American/Alaska Native':1,'White':2, 'Black':3, 'Asian/Pacific Islander':4})
d['Relationship']=d['Relationship'].map({'Acquaintance':0, 'Unknown':-1, 'Wife':2, 'Stranger':3, 'Girlfriend':4, 'Ex-Husband':5, 'Brother':6, 'Stepdaughter':7, 'Husband':8, 'Sister':9, 'Friend':10, 'Family':11, 'Neighbor' :12,'Father' :13,'In-Law':14, 'Son':15, 'Ex-Wife':16, 'Boyfriend':18, 'Mother':17 ,'Common-Law Husband':19, 'Common-Law Wife':20, 'Stepfather':21, 'Stepson':22 ,'Stepmother':23 ,'Daughter':24 ,'Boyfriend/Girlfriend':25 ,'Employer' :26,'Employee':27})
d['Crime Type']=d['Crime Type'].map({'Murder or Manslaughter':2, 'Manslaughter by Negligence':1})
d_nonan=d.copy()
d = d.applymap(lambda x: np.nan if x==-1 else x)

colormap = plt.cm.viridis
plt.figure(figsize=(12,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(d.astype(float).corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)

Crime solving rate
=====================

by perpetrator race
-------------------

The table below shows the crime solving rate by perpetrator race. 
Obviously, when the crime is not solved, the perp race is unknown. This does not help much.

In [3]:
d2 = df[['Perpetrator Race','Crime Solved','Year']].groupby(by=['Perpetrator Race','Crime Solved']).agg({'Year':'count'})
print(d2.groupby(level=0).apply(lambda x:100 * x/float(x.sum())))

by Victim race
-------------

Let's now have a look at the link between the crime solving rate and the victim race.
The chart below shows that the solving rate for all victim races is globally upwards, except for black victims. In this group, the solving rate is decreasing.

In [4]:
d4= df[['Victim Race','Year','Crime Solved']]
d4=d4.groupby(by=['Victim Race', 'Year','Crime Solved']).agg({'Crime Solved':'count'})
d4= d4.groupby(level=[0,1]).apply(lambda x:100 * x/x.sum())
fig = plt.figure()
ax = fig.add_subplot(111)
d4.columns=["Solved_count"]

for name, group in d4.groupby(level=0):
    if name!="Unknown":
        group= group.reset_index().drop("Victim Race",1)
        group=group[group["Crime Solved"]=="Yes"]
        ax.plot(group["Year"], group["Solved_count"], label=name)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels)
plt.title('Crime solving rate by victim race')
plt.show()

by state
--------

In [5]:
d5 = df[['State','Crime Solved','Year']].groupby(by=['State','Crime Solved']).agg({'Year':'count'})
d5=d5.groupby(level=0).apply(lambda x:100 * x/float(x.sum())).reset_index()
d5 = d5[d5['Crime Solved']=="Yes"].drop("Crime Solved",1)
# Convert state names to their abbreviations.
states_dict = {'Alaska':'AK','Alabama':'AL','Arkansas':'AR','Arizona':'AZ', 'California':'CA', 'Colorado':'CO',         'Connecticut':'CT',         'District of Columbia':'DC',         'Delaware':'DE',  'Florida':'FL',         'Georgia':'GA',         'Hawaii':'HI',         'Iowa':'IA',         'Idaho':'ID',         'Illinois':'IL',         'Indiana':'IN',         'Kansas':'KS',         'Kentucky':'KY',         'Louisiana':'LA',         'Massachusetts':'MA',         'Maryland':'MD',         'Maine':'ME',         'Michigan':'MI',         'Minnesota':'MN',         'Missouri':'MO',         'Mississippi':'MS',         'Montana':'MT',         'North Carolina':'NC',         'North Dakota':'ND',         'Nebraska':'NE',         'New Hampshire':'NH',         'New Jersey':'NJ',         'New Mexico':'NM',         'Nevada':'NV',         'New York':'NY',         'Ohio':'OH',         'Oklahoma':'OK',         'Oregon':'OR',         'Pennsylvania':'PA',         'Puerto Rico':'PR',         'Rhode Island':'RI',         'South Carolina':'SC',         'South Dakota':'SD',         'Tennessee':'TN',         'Texas':'TX',         'Utah':'UT',         'Virginia':'VA',         'Vermont':'VT',         'Washington':'WA',         'Wisconsin':'WI',         'West Virginia':'WV',   'Wyoming':'WY'};
d5['State'].replace(states_dict,inplace = True);
# show terrorism in each state on a map
init_notebook_mode()
# plotly code for choropleth map
scale = [[0, 'rgb(229, 239, 245)'],[1, 'rgb(1, 97, 156)']]

data = [ dict(
        type = 'choropleth',
        colorscale = scale,
        autocolorscale = False,
        showscale = True,
        locations = d5['State'],
        z = d5['Year'],
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255, 255, 255)',
                width = 2
            ) ),
        ) ]
layout = dict(
        title = 'Crime solving rate per state',
        geo = dict(
            scope = 'usa',
            projection = dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
            countrycolor = 'rgb(255, 255, 255)')        
             )
 
figure = dict(data=data, layout=layout)
iplot(figure)

In [6]:
d6= df[['Year','State','Crime Solved']]
d6=d6.groupby(by=['Year','State','Crime Solved']).agg({'Crime Solved':'count'})
d6.columns=["Solved_count"]
d6= d6.groupby(level=[0,1]).apply(lambda x:100 * x/x.sum()).reset_index()
d6 = d6[d6['Crime Solved']=="Yes"].drop("Crime Solved",1)
years=np.sort(d6["Year"].unique())
d6['State'].replace(states_dict,inplace = True);

init_notebook_mode()
# plotly code for choropleth map
scale = [[0, 'rgb(229, 239, 245)'],[1, 'rgb(1, 97, 156)']]
data = [ dict(
        type = 'choropleth',
        colorscale = scale,
        autocolorscale = False,
        showscale = True,
        locations = d6[d6['Year']=='1980']['State'],
        z = d6[d6['Year']=="1980"]["Solved_count"],
        locationmode = 'USA-states',
        marker = dict(line = dict (color = 'rgb(255, 255, 255)',width = 2) ),
        ) ]
layout = dict(
            title = 'Crime solving rate per state',
            geo = dict(
                scope = 'usa',
                projection = dict( type='albers usa' ),
                showlakes = True,
                lakecolor = 'rgb(255, 255, 255)',
                countrycolor = 'rgb(255, 255, 255)'),
                updatemenus = [{'type' : 'buttons','buttons' : [{ 'label':'Play','method':'Animate','args': [None]}]}]
            )
    
frames=[dict(data=[dict(
    type='choropleth',
    locations = [d6[d6['Year']==year]['State'] for year in years],
    z=[d6[d6["Year"]==year]["Solved_count"] for year in years])])]  

frames= [{'data': [{'type':'choropleth',
                    'colorscale' :'scale',
                    'autocolorscale' : 'False',
                    'showscale' : 'True',
                    'locationmode' : 'USA-states',
                    'marker' : {'line' : {'color' : 'rgb(255, 255, 255)','width' : 2}},
                    'z': d6[d6["Year"]=="1980"]["Solved_count"],
                    'locations': d6[d6['Year']=="1980"]['State']   
                   }],
          'data':[{'type':'choropleth',
                    'colorscale' :'scale',
                    'autocolorscale' : 'False',
                    'showscale' : 'True',
                    'locationmode' : 'USA-states',
                    'marker' : {'line' : {'color' : 'rgb(255, 255, 255)','width' : 2}},
                    'z': d6[d6["Year"]=="1981"]["Solved_count"],
                    'locations': d6[d6['Year']=="1981"]['State']   
                   }],
          'data':[{'type':'choropleth',
                    'colorscale' :'scale',
                    'autocolorscale' : 'False',
                    'showscale' : 'True',
                    'locationmode': 'USA-states',
                    'marker' : {'line' : {'color' : 'rgb(255, 255, 255)','width' : 2}},
                    'z': d6[d6["Year"]=="1982"]["Solved_count"],
                     'locations': d6[d6['Year']=="1982"]['State']   
                   }]
         }]
    
figure = dict(data=data, layout=layout, frames=frames)
iplot(figure)

Link between perpetrator and victim race
========================================

Here we have a look at the correlation of perpetrator and victim races.
What is observable is that perpetrators tend to choose victim of their own racial group.

In [7]:
d4 = df[['Victim Race','Perpetrator Race','Year']].groupby(by=['Victim Race','Perpetrator Race']).agg({'Year':'count'})
d4= d4.groupby(level=0).apply(lambda x:100 * x/float(x.sum()))

fig = plt.figure()
ax = fig.add_subplot(111)

victim = d4.index.get_level_values(0).unique()

ind = np.arange(5)
plt.xticks(ind, victim)
width = 0.35
d4 = d4.reset_index()
s2=[0,0,0,0,0]
colors=['yellow','black','red','grey','white']
i=0
for r in victim:
    s =d4[d4['Perpetrator Race']==r]["Year"]
    p= ax.bar(ind,s, width, bottom=s2, color=colors[i], label=r)
    i=i+1
    s2 = [x + y for x, y in zip(s, s2)]
    
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('Victims')
plt.show()

In [8]:
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
np.random.seed(5)

d_nonan = d_nonan[['Relationship','Victim Race','Weapon',]]
d_nonan=d_nonan.dropna()

est=KMeans(n_clusters=3).fit(d_nonan)
labels = est.labels_
print(labels)

In [11]:
fignum=1
fig = plt.figure(fignum, figsize=(10, 10))
color_labels={}
color_map={0:"red",1:"green",2:"blue"}
for r in labels:
    color_labels[r]=color_map[labels[r]]
    print(r)

print(labels[1])
print(color_labels)


ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(d_nonan['Relationship'], d_nonan['Victim Race'],d_nonan['Weapon'], 
           c=color_labels)
ax.set_xlabel('Relationship')
ax.set_ylabel('Victim Race')
ax.set_zlabel('Weapon')
ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.set_title(fignum)

In [10]:
fignum = 1

fig = plt.figure(fignum, figsize=(10, 10))

ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
use_colours = {0: "red", 1: "green", 2: "blue"}
d_nonan['Weapon']=d_nonan['Weapon'].map({0:'Blunt Object', 1:'Strangulation', -1:'Unknown', 3:'Rifle', 4:'Knife', 5:'Firearm', 6:'Shotgun', 7:'Fall', 8:'Handgun', 9:'Drowning', 10:'Suffocation', 11:'Explosives', 12:'Fire',13:'Drugs', 14:'Gun', 15:'Poison'})
ax.scatter(d_nonan['Relationship'], d_nonan['Victim Race'],d_nonan['Weapon'], 
           c=[use_colours[x] for x in labels])
ax.set_xlabel('Relationship')
ax.set_ylabel('Victim Race')
ax.set_zlabel('Weapon')
#ax.get_zlabel().map({0:'Blunt Object', 1:'Strangulation', -1:'Unknown', 3:'Rifle', 4:'Knife', 5:'Firearm', 6:'Shotgun', 7:'Fall', 8:'Handgun', 9:'Drowning', 10:'Suffocation', 11:'Explosives', 12:'Fire',13:'Drugs', 14:'Gun', 15:'Poison'})

#ax.w_xaxis.set_ticklabels([])
#ax.w_yaxis.set_ticklabels([])
ax.set_title(fignum)